In [ ]:
!git clone https://github.com/Farhan9801/Farhan9801-arhan9801-image-enhancer_2.git

In [ ]:
!pip install opencv-python
!pip install timm
!pip install pandas

In [ ]:
import pandas as pd

total_urls = pd.read_csv('image_url_ideabook_page_1-20_total_1000.csv')
total_urls = total_urls['ImageURL'].tolist()


In [ ]:
urls = total_urls[25:200]

In [ ]:
import requests
import os

output_dir = 'latest_download'
os.makedirs(output_dir, exist_ok=True)

for i, url in enumerate(urls):
    try:
        print(i, url)
        data = requests.get(url)
        if data.status_code == 200:
            file_name = os.path.basename(url)
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'wb') as file:
                file.write(data.content)

    except Exception as e:
        print(f'Unable to get the image due to {e}')

# model download

In [ ]:
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P model_zoo/swinir

In [ ]:
from main_test_swinir import main
import argparse

args = argparse.Namespace(
    task='real_sr',
    scale=4,
    noise=15,
    jpeg=40,
    training_patch_size=128,
    large_model=True,
    model_path='model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth',
    folder_lq='testsets/RealSRSet+5images',
    folder_gt=None,
    tile=None,
    tile_overlap=32
)

main(args)


In [ ]:
import os
import shutil
from PIL import Image

# === Configuration ===
SOURCE_FOLDER = "/Users/betterhalf/Farhan/tt/Original"
FINAL_FOLDER = "/Users/betterhalf/Farhan/tt/final_folder"
MIDDLE_FOLDER = '/Users/betterhalf/Farhan/tt/middle_folder'
MAX_SIZE_MB = 1
QUALITY = 80  # Adjust this for compression level (60–80 is a good range)

def ensure_backup_folder_structure(src_path, base_src, base_backup):
    relative_path = os.path.relpath(src_path, base_src)
    backup_path = os.path.join(base_backup, relative_path)
    os.makedirs(os.path.dirname(backup_path), exist_ok=True)
    return backup_path

def compress_and_replace_image(file_path, final_path, middle_path):
    QUALITY = 80
    try:
        with Image.open(file_path) as img:
            img_format = img.format
            if img.format.upper() not in ['JPEG', 'PNG', 'BMP', 'GIF', 'TIFF']:
                print(f"New Format detected {file_path} (Detected format: {img.format})")
                if img.format.upper() == 'MPO':
                    file_size_bytes = os.path.getsize(file_path)
                    file_size_mb = file_size_bytes / (1024 * 1024)
                    print(f'BEFORE_compression {file_size_mb}')
                    img.seek(0)  # Use the first frame
                    img = img.convert('RGB')
                    while file_size_mb > 1 and  QUALITY >= 60:
                        img.save(middle_path, format='WEBP', quality=QUALITY, optimize=True, progressive=True)
                        file_size_bytes = os.path.getsize(middle_path)
                        file_size_mb = file_size_bytes / (1024 * 1024)
                        print(f'after_compression {file_size_mb}')
                        QUALITY -= 5

                    shutil.copy2(middle_path, final_path)  # copy2 also preserves metadata
                    return

            # Move original image to backup folder
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = file_size_bytes / (1024 * 1024)
            print(f'BEFORE_compression {file_size_mb}')
            # shutil.copy2(file_path, backup_path)
            # print(f"Backed up: {file_path} -> {backup_path}")
            # Compress and overwrite the image
            while file_size_mb > 1 and  QUALITY >= 60:
                file_size_bytes = os.path.getsize(file_path)
                file_size_mb = file_size_bytes / (1024 * 1024)
                print(f'BEFORE_compression {file_size_mb}')
                img.save(middle_path, format='WEBP', quality=QUALITY, optimize=True, progressive=True)
                file_size_bytes = os.path.getsize(middle_path)
                file_size_mb = file_size_bytes / (1024 * 1024)
                print(f'after_compression {file_size_mb}')
                QUALITY -= 5

            shutil.copy2(middle_path, final_path)  # copy2 also preserves metadata
            return
            # img.save(final_path, format='WEBP', quality=QUALITY, optimize=True, progressive=True)
            # file_size_bytes = os.path.getsize(file_path)
            # file_size_mb = file_size_bytes / (1024 * 1024)
            # print(f'after_compression {file_size_mb}')
            # print(f"Compressed and replaced: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

def is_large_image(file_path):
    return os.path.getsize(file_path) > MAX_SIZE_MB * 1024 * 1024

def process_images(src_folder, final_folder, middle_folder):
    for root, _, files in os.walk(src_folder):
        for name in files:
            file_path_orinal = os.path.join(root, name)
            middle_path = os.path.join(middle_folder, name)
            if is_large_image(file_path_orinal):
                # Change extension to .webp for output paths
                final_path = ensure_backup_folder_structure(file_path_orinal, src_folder, final_folder)
                final_path = os.path.splitext(final_path)[0] + '.webp'

                middle_path = ensure_backup_folder_structure(file_path_orinal, src_folder, middle_folder)
                middle_path = os.path.splitext(middle_path)[0] + '.webp'

                print(middle_path)
                compress_and_replace_image(file_path_orinal, final_path, middle_path)

if __name__ == "__main__":
    if not os.path.isdir(SOURCE_FOLDER):
        print(f"Source folder does not exist: {SOURCE_FOLDER}")
    if not os.path.isdir(FINAL_FOLDER):
        os.makedirs(FINAL_FOLDER)
        print(f"Created backup folder: {FINAL_FOLDER}")
    if not os.path.isdir(MIDDLE_FOLDER):
        os.makedirs(MIDDLE_FOLDER)
        print('Created the middle folder')

    process_images(SOURCE_FOLDER, FINAL_FOLDER, MIDDLE_FOLDER)